In [5]:
import glob
import pandas as pd
import numpy as np
import re
import pycountry
from fuzzywuzzy import fuzz

K:\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [58]:
csv_files = glob.glob('./data_folder/*.csv')
dataframes=[]
col_names_list = []
for x in csv_files:
    dataframes.append(pd.read_csv(x, low_memory = False))
    col_names_list.append(list(dataframes[-1].columns))

In [59]:
#checks to see if the df has anything to signify country
[[csv_files[y],sum([bool(re.search('ountry',x)) for x in col_names_list[y]])>0] for y in range(len(col_names_list))]

[['./data_folder\\city_temperature.csv', True],
 ['./data_folder\\EdStatsData.csv', True],
 ['./data_folder\\global-city-population-estimates-CITIES-OVER-300K.csv',
  True],
 ['./data_folder\\Main_df.csv', True],
 ['./data_folder\\Qol_data.csv', False],
 ['./data_folder\\WPP2019_PopulationByAgeSex_Medium.csv', False],
 ['./data_folder\\WRP national data.csv', False]]

In [10]:
#List showing what each of the data frames and their variable names
Temp_df = dataframes[0].copy() # has temperature data from 'cty_temperature.csv'
Ed_df = dataframes[1].copy() # has Education data from 'EdStatsData.csv'
City_pop_df = dataframes[2].copy() # has population data from 'global-city-population-estimates-CITIES-OVER-300K.csv'
QoL_df = dataframes[3].copy() # has Quality of Life data from 'Qol_data.csv'
Age_df = dataframes[4].copy() # has Age demographics data from 'WPP2019_PopulationByAgeSex_Medium.csv'
Rel_df = dataframes[5].copy() # has religion data from 'WRP national data.csv'

<h1> Temperature Data Frame: </h1>

In [11]:
Temp_df.head()

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [48]:
Temp_df = dataframes[0].copy() #Recopies so if the cell is run alone it still works
Temp_df = Temp_df.drop(['Month','Day','Year'],axis = 1)
Temp_df = Temp_df.drop('Region',axis = 1)
Temp_df.fillna('None',inplace = True)
Temp_df = Temp_df.groupby(['Country', 'State', 'City']).mean().reset_index() #averages all historic temperatures by country and city
Temp_df.head()

,Country,State,City,AvgTemperature
0,Albania,None,Tirana,33.172923
1,Algeria,None,Algiers,63.755439
2,Argentina,None,Buenos Aires,62.304900
3,Australia,None,Brisbane,68.072491
4,Australia,None,Canberra,55.579689


<h1> Education Data Frame: </h1>

In [15]:
Ed_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,54.822121,54.894138,56.209438,57.267109,57.991138,59.36554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
Ed_df = dataframes[1].copy() #Recopies so if the cell is run alone it still works

#Creates a List of different education data set types that include the full population
ed_types = []
for x in dataframes[1]['Indicator Name'].unique():
    older_t_15_test = (x.find('15+') > -1)
    gender_test = (x.lower().find('female')==-1) & (x.lower().find('male')==-1)
    data_set_test = ((x.find('Barro') >-1) | (x.find('Witt') >-1)) & (x.lower().find('percent')>-1)
    if older_t_15_test & gender_test & data_set_test:
        ed_types.append(x)

#only selects education data values for data in the select data sets
Ed_df = Ed_df[Ed_df['Indicator Name'].isin(ed_types)]

#removes data outside 2012-2021 the time frame for where Qol data is in
col = [colm for colm in Ed_df.columns[4:-1] if (int(colm) > 2011 and int(colm) < 2022)]
col.extend(Ed_df.columns[:3])
Ed_df = Ed_df[col].reset_index(drop = True)

#unpivots the year data
Ed_df = Ed_df.melt(id_vars = ['Country Name', 'Country Code', 'Indicator Name'], var_name = 'Year', value_name = 'Percent')
Ed_df = Ed_df[~np.array(Ed_df['Percent'].isna())].reset_index(drop = True)

#Selects the max percentage for each education type divded
#into three categories less than secondary, some secondary, and some tertiary
ed_score_list = []
for name in Ed_df['Indicator Name']:
    if (name.lower().find('tertiary') > -1) | (name.lower().find('post secondary') > -1):
        ed_score_list.append('pct_some_ter')
    elif name.lower().find('secondary') > -1:
        ed_score_list.append('pct_some_sec')
    else:
        ed_score_list.append('pct_lt_sec')
Ed_df['Education Score']=ed_score_list
Ed_df = Ed_df.drop('Indicator Name', axis = 1)
Ed_df = Ed_df.groupby(['Country Name', 'Country Code', 'Year','Education Score']).max().reset_index()
Ed_df = Ed_df.pivot(index = ['Country Name', 'Country Code', 'Year'], columns = 'Education Score', values = 'Percent').reset_index()


Ed_df = Ed_df.drop('Country Name',axis = 1).sort_values('Country Code').reset_index(drop = True)
col = []
col.append('Country Code')
col.extend(Ed_df.columns[1:])
Ed_df.columns = col

#Takes the mean over the time period
Ed_df = Ed_df.drop('Year', axis = 1)
Ed_df = Ed_df.groupby('Country Code').mean().reset_index()
Ed_df.head()

,Country Code,pct_lt_sec,pct_some_sec,pct_some_ter
0,ABW,0.200,0.330,0.230
1,ALB,0.055,0.435,0.085
2,ARE,0.115,0.355,0.175
3,ARG,0.250,0.330,0.135
4,ARM,0.020,0.665,0.215


<h1> Population Data Frame: </h1>

In [17]:
City_pop_df.head()

,Country Code,Country or area,City Code,Urban Agglomeration,Note,Latitude,Longitude,1950,1955,1960,...,1985,1990,1995,2000,2005,2010,2015,2020,2025,2030
0,392,Japan,21671,Tokyo,325.0,35.689500,139.691710,"11,275","13,713","16,679",...,"30,304","32,530","33,587","34,450","35,622","36,834","38,001","38,323","37,876","37,190"
1,356,India,21228,Delhi,318.0,28.666670,77.216670,"1,369","1,782","2,283",...,"7,325","9,726","12,407","15,732","18,670","21,935","25,703","29,348","32,727","36,060"
2,156,China,20656,Shanghai,202.0,31.220000,121.460000,"4,301","5,846","6,820",...,"6,847","7,823","10,450","13,959","16,763","19,980","23,741","27,137","29,442","30,751"
3,76,Brazil,20287,São Paulo,NaN,-23.550000,-46.640000,"2,334","3,044","3,970",...,"13,395","14,776","15,913","17,014","18,288","19,660","21,066","22,119","22,899","23,444"
4,356,India,21206,Mumbai (Bombay),NaN,19.073975,72.880838,"2,857","3,432","4,060",...,"10,391","12,436","14,310","16,367","17,891","19,422","21,043","22,838","25,207","27,797"


In [18]:
City_pop_df = dataframes[2].copy() #Recopies so if the cell is run alone it still works

#Filters out unneeded years
col = [City_pop_df.columns[1],City_pop_df.columns[3]]
col.extend([x for x in City_pop_df.columns[7:-1] if (int(x)>2004) & (int(x)<2021)])
City_pop_df = City_pop_df[col]

#unpivots the dataframe
City_pop_df = City_pop_df.melt(id_vars = ['Country or area', 'Urban Agglomeration'], var_name = 'Year', value_name = 'Pop in Thousands')

#Creates a new data column to find unique city country combinations
City_pop_df['Country City'] = City_pop_df['Country or area'] + ',' + City_pop_df['Urban Agglomeration']
city_list = list(City_pop_df['Country City'].str.split(','))

#creates a dictionary for each year to store the population for that year in that specific city
pop_2005 = {row['Country City'] : row['Pop in Thousands'] for idx,row in City_pop_df.iterrows() if row['Year'] == '2005'}
pop_2010 = {row['Country City'] : row['Pop in Thousands'] for idx,row in City_pop_df.iterrows() if row['Year'] == '2010'}
pop_2015 = {row['Country City'] : row['Pop in Thousands'] for idx,row in City_pop_df.iterrows() if row['Year'] == '2015'}
pop_2020 = {row['Country City'] : row['Pop in Thousands'] for idx,row in City_pop_df.iterrows() if row['Year'] == '2020'}

#removes data from 2005 since we only need it to estimate population growth and it is stored in pop_2005.
City_pop_df = City_pop_df[City_pop_df['Year'].agg(int) > 2005]

#uses the data from 2005 to 2020 to get population growth data
pop_list = []
pop_growth_list = []
country_city = []
for idx, row in City_pop_df.iterrows():
    yr = int(row['Year'])
    p2005 = int(pop_2005[row['Country City']].replace(',',''))
    p2010 = int(pop_2010[row['Country City']].replace(',',''))
    p2015 = int(pop_2015[row['Country City']].replace(',',''))
    p2020 = int(pop_2020[row['Country City']].replace(',',''))
    if yr == 2010:
        pop_growth = (p2010 - p2005)
        pop = p2010
    elif yr == 2015:
        pop_growth = (p2015 - p2010)
        pop = p2015
    else:
        pop_growth = (p2020 - p2015)
        pop = p2020
    pop_growth_list.append(pop_growth)
    pop_list.append(pop)
    country_city.append(row['Country City'])
City_pop_df['Pop in Thousands'] = pop_list
City_pop_df['Pop growth in Thousands'] = pop_growth_list

City_pop_df = City_pop_df.drop('Country City',axis = 1)
City_pop_df.columns = ['Country','City', 'Year', 'pop_thou','avg_5_yr_growth_thou']
City_pop_df = City_pop_df.drop('Year', axis = 1)
City_pop_df = City_pop_df.groupby(['City','Country']).mean().reset_index()


City_pop_df['pop_thou'] = City_pop_df['pop_thou'].agg(int)
City_pop_df['avg_5_yr_growth_thou'] = City_pop_df['avg_5_yr_growth_thou'].agg(int)
City_pop_df.head()

,City,Country,pop_thou,avg_5_yr_growth_thou
0,Aba,Nigeria,962,132
1,Abakaliki,Nigeria,447,103
2,Abeokuta,Nigeria,506,45
3,Abidjan,Côte d'Ivoire,4919,734
4,Abomey-Calavi,Benin,764,203


<h1> Quality of Life Data Frame: </h1>

In [19]:
QoL_df.head()

,year,City,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index
0,2012-Q1,"Berlin, Germany",215.25,140.62,77.34,64.81,89.06,5.74,23.00,40.00
1,2012-Q1,"Zurich, Switzerland",213.18,136.31,83.85,77.78,166.07,8.27,27.33,26.50
2,2012-Q1,"Edmonton, Canada",204.86,124.69,58.98,78.89,112.10,3.59,35.33,21.25
3,2012-Q1,"Perth, Australia",196.00,118.34,48.75,87.04,142.18,5.24,27.00,15.00
4,2012-Q1,"Calgary, Canada",190.78,124.25,71.88,70.42,111.71,4.58,62.00,25.08


In [20]:
QoL_df = dataframes[3].copy() #Recopies so if the cell is run alone it still works

#Splits City and Country Data into two columns
city_list = list(dataframes[3].copy()['City'].str.split(','))
country_list = []
city_list_df = []
state_list_df = []
for city in city_list:
    country_list.append(city[-1])
    city_list_df.append(city[0])
    if len(city) == 3:
        state_list_df.append(city[1])
    else:
        state_list_df.append('None')
        
QoL_df['Country'] = country_list
QoL_df['City'] = city_list_df
QoL_df['State'] = state_list_df
QoL_df = QoL_df.groupby(['City','Country','State']).mean().reset_index()
QoL_df.head()

,City,Country,State,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index
0,Abu Dhabi,United Arab Emirates,None,169.568000,117.263333,86.409333,62.617333,62.433333,5.294000,27.154000,51.220667
1,Adelaide,Australia,None,199.101765,118.463529,64.796471,71.123529,84.723529,5.124118,27.401176,21.015294
2,Ahmedabad,India,None,113.272143,54.841429,69.215714,68.046429,26.800714,8.554286,41.815714,70.927857
3,Albuquerque,United States,NM,172.184286,116.474286,31.161429,67.892857,64.925714,2.854286,27.110000,23.460000
4,Almaty,Kazakhstan,None,90.724167,42.469167,41.099167,54.720000,34.263333,12.137500,32.410000,77.376667


<h1> Age Demographics Data Frame: </h1>

In [21]:
Age_df.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,AgeGrp,AgeGrpStart,AgeGrpSpan,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,1950,1950.5,0-4,0,5,630.044,661.578,1291.622
1,4,Afghanistan,2,Medium,1950,1950.5,5-9,5,5,516.206,487.335,1003.541
2,4,Afghanistan,2,Medium,1950,1950.5,10-14,10,5,461.378,423.326,884.704
3,4,Afghanistan,2,Medium,1950,1950.5,15-19,15,5,414.369,369.363,783.732
4,4,Afghanistan,2,Medium,1950,1950.5,20-24,20,5,374.109,318.392,692.501


In [22]:
Age_df = dataframes[4].copy() #Recopies so if the cell is run alone it still works

Age_df['Country'] = Age_df['Location']

#Gets percent population of each age group
Age_df_m = Age_df[['Location', 'Time', 'PopTotal']].groupby(by = ['Location','Time'],as_index = False).sum().reset_index()
Age_df_m = Age_df.merge(Age_df_m, on = ['Location', 'Time'], how = 'left')
Age_df_m['AgeGrpPct'] = 100* Age_df_m['PopTotal_x']/Age_df_m['PopTotal_y']

Age_df_m = Age_df_m[['Country', 'Time', 'AgeGrp','AgeGrpPct']]
Age_df = Age_df_m
Age_df = Age_df[(Age_df['Time']>2011) & (Age_df['Time']<2021)].reset_index(drop = True)

#gets percent of population that is fertile (ages 15 - 50)
fert_list=[]
for grp in  Age_df['AgeGrp']:
    oldest = int(str(grp).split('-')[-1].replace('+',''))
    if (oldest>14) & (oldest<50):
        fert_list.append(1)
    else:
        fert_list.append(0)
Age_df['Fertile'] = fert_list
Age_df = Age_df.drop('AgeGrp', axis = 1)
Age_df = Age_df.groupby(by = ['Country','Fertile','Time']).sum().reset_index()
Age_df = Age_df[Age_df['Fertile'] == 1].drop('Fertile',axis = 1).reset_index(drop = True)

Age_df = Age_df.drop('Time', axis = 1)
Age_df = Age_df.groupby(by = ['Country']).mean().reset_index()
col = list(Age_df.columns[:-1])
col.append('Percent Fertile')

Age_df.columns = col
Age_df.head()

,Country,Percent Fertile
0,Afghanistan,47.066946
1,Africa,48.129097
2,African Group,48.125125
3,African Union,48.129770
4,African Union: Central Africa,45.508465


<h1> Religion Demographics Data Frame: </h1>

In [23]:
col = Rel_df.columns
print([name for name in col])

['year', 'state', 'name', 'chrstprot', 'chrstcat', 'chrstorth', 'chrstang', 'chrstothr', 'chrstgen', 'judorth', 'jdcons', 'judref', 'judothr', 'judgen', 'islmsun', 'islmshi', 'islmibd', 'islmnat', 'islmalw', 'islmahm', 'islmothr', 'islmgen', 'budmah', 'budthr', 'budothr', 'budgen', 'zorogen', 'hindgen', 'sikhgen', 'shntgen', 'bahgen', 'taogen', 'jaingen', 'confgen', 'syncgen', 'anmgen', 'nonrelig', 'othrgen', 'sumrelig', 'pop', 'chrstprotpct', 'chrstcatpct', 'chrstorthpct', 'chrstangpct', 'chrstothrpct', 'chrstgenpct', 'judorthpct', 'judconspct', 'judrefpct', 'judothrpct', 'judgenpct', 'islmsunpct', 'islmshipct', 'islmibdpct', 'islmnatpct', 'islmalwpct', 'islmahmpct', 'islmothrpct', 'islmgenpct', 'budmahpct', 'budthrpct', 'budothrpct', 'budgenpct', 'zorogenpct', 'hindgenpct', 'sikhgenpct', 'shntgenpct', 'bahgenpct', 'taogenpct', 'jaingenpct', 'confgenpct', 'syncgenpct', 'anmgenpct', 'nonreligpct', 'othrgenpct', 'sumreligpct', 'total', 'dualrelig', 'datatype', 'sourcereliab', 'recreliab

In [24]:
#Recopies so if the cell is run alone it still works
Rel_df = dataframes[5].copy()[dataframes[5].copy()['year'] == 2010].reset_index(drop = True)

col = Rel_df.columns
col2 = col[0:3:2].append(Rel_df.columns[40:75])
Rel_df = Rel_df[col2]
#prefs is a list of the prefixes for each religion type to pull out only percentage
prefs = ['chrst', 'jud', 'islm','bud', 'zor', 'hin', 'sikh' ,'shnt', 'bah', 'tao', 'jain', 'con', 'syn','anm','non','othr','sum']
Rel_df2 = Rel_df.copy()
for pref in prefs:
    Rel_df2[pref + 'pct'] = np.zeros(len(Rel_df2))
    for name in col2:
        if name[0:len(pref)] == pref:
            Rel_df2[pref + 'pct'] = Rel_df2[pref + 'pct'] + Rel_df2[name]
            Rel_df2 = Rel_df2.drop(name, axis = 1)
Rel_df = Rel_df2.copy()
Rel_df = Rel_df.drop('year', axis = 1)

Rel_df.groupby('name').mean()
Rel_df['Country Code'] = Rel_df['name']
Rel_df = Rel_df.drop('name', axis = 1)
Rel_df.head()

,chrstpct,judpct,islmpct,budpct,zorpct,hinpct,sikhpct,shntpct,bahpct,taopct,jainpct,conpct,synpct,anmpct,nonpct,othrpct,sumpct,Country Code
0,1.4908,0.0381,0.0180,0.0218,0.0053,0.0057,0.0013,0.0005,0.0015,0.0000,0.0003,0.0003,0.0026,0.0057,0.1900,0.0025,0.0,USA
1,1.5322,0.0199,0.0388,0.0388,0.0002,0.0080,0.0080,0.0000,0.0005,0.0001,0.0001,0.0001,0.0008,0.0021,0.1643,0.0010,0.0,CAN
2,1.9320,0.0020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0003,0.0000,0.0000,0.0000,0.0032,0.0290,0.0005,0.0,BHM
3,1.3178,0.0002,0.0014,0.0000,0.0000,0.0022,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5000,0.0000,0.1315,0.0000,0.0,CUB
4,1.6400,0.0000,0.0004,0.0000,0.0000,0.0000,0.0000,0.0000,0.0009,0.0000,0.0000,0.0000,0.4500,0.0000,0.1000,0.0000,0.0,HAI


In [50]:
def get_country_codes(df,lbl):
    list_alpha_3 = [i.alpha_3 for i in list(pycountry.countries)]
    list_country = [i.name for i in list(pycountry.countries)]
    codes = []
    for cntr in list(df[lbl]):
        for idx in range(len(list_country)):
            if fuzz.ratio(cntr.strip().lower(),list_country[idx].lower())>95:
                code = list_alpha_3[idx]
                break
            else:
                code = pd.NA
        codes.append(code)
    return codes

country_list = []
for country in Temp_df['Country']:
    if country.strip() == 'US':
        country_list.append('United States')
    else:
        country_list.append(country.strip())
Temp_df['Country'] = country_list     
Temp_df['Country Code'] = get_country_codes(Temp_df,'Country')

Age_df['Country Code'] = get_country_codes(Age_df,'Country')
Age_df = Age_df[~np.array(Age_df['Country Code'].isna())]

QoL_df['Country Code'] = get_country_codes(QoL_df,'Country')
QoL_df['Country'] = QoL_df['Country'].str.strip()
City_pop_df['Country Code'] = get_country_codes(City_pop_df,'Country')

In [53]:
def get_city_name(lis,city):
    tst = city.strip()
    ratios = []
    for test_city in lis:
        tst2 = test_city.strip()
        min_ind = min(len(tst),len(tst2))
        ratios.append(fuzz.ratio(tst[:min_ind].lower().strip(),tst2[:min_ind].lower().strip()))
    rmax = max(ratios)
    if rmax>80:
        return lis[ratios.index(rmax)]
    else:
        return 'None'

cities = list(QoL_df['City'].unique())
out_list = []
[out_list.append(get_city_name(cities,city)) for city in list(City_pop_df['City'])]  
City_pop_df['City'] = out_list

Main_df = QoL_df.merge(City_pop_df, on = ['City', 'Country Code'], how = 'left')
Main_df['Country'] = Main_df['Country_x']
Main_df = Main_df.drop(['Country_x','Country_y'], axis = 1)

Main_df = Main_df.merge(Age_df, on = 'Country Code', how = 'left')
Main_df = Main_df.drop(['Country_x','Country_y'], axis = 1)

Main_df = Main_df.merge(Rel_df, on = 'Country Code', how = 'left')

cities = list(QoL_df['City'].unique())
out_list = []
[out_list.append(get_city_name(cities,city)) for city in list(Temp_df['City'])]  
Temp_df['City'] = out_list

Main_df = Main_df.merge(Temp_df, on = ['Country Code', 'State'], how = 'left')

Main_df = Main_df.merge(Ed_df, on = 'Country Code', how = 'left')
Main_df.dropna(inplace = True)
Main_df.reset_index(inplace = True, drop = True)
Main_df.to_csv('.//data_folder//Main_df.csv')

In [54]:
for s in Main_df.columns:
    print(Main_df[s].isnull().describe())

count       275
unique        1
top       False
freq        275
Name: City_x, dtype: object
count       275
unique        1
top       False
freq        275
Name: State, dtype: object
count       275
unique        1
top       False
freq        275
Name: Quality of Life Index, dtype: object
count       275
unique        1
top       False
freq        275
Name: Purchasing Power Index, dtype: object
count       275
unique        1
top       False
freq        275
Name: Safety Index, dtype: object
count       275
unique        1
top       False
freq        275
Name: Health Care Index, dtype: object
count       275
unique        1
top       False
freq        275
Name: Cost of Living Index, dtype: object
count       275
unique        1
top       False
freq        275
Name: Property Price to Income Ratio, dtype: object
count       275
unique        1
top       False
freq        275
Name: Traffic Commute Time Index, dtype: object
count       275
unique        1
top       False
freq        275
Nam